## Install and import necessary packages

In [36]:
!pip install geemap --quiet
!pip install earthengine-api --quiet
!pip install PyCRS --quiet
!pip install restee --quiet
!pip install config --quiet
!pip install geojson --quiet
!pip install jupyter-leaflet --quiet

import geemap, ee, os, sys, pycrs, datetime
import geemap.colormaps as cm

from ipyleaflet import *
from ipywidgets import Label
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import config
import json
import csv
import json
import geojson
import geopandas as gpd
import rasterio
import importlib
import numpy as np

import requests
import zipfile
import os
from os import listdir
from os.path import isfile,join

## Log in to your own Google Earth Engine account

In [37]:
ee.Authenticate()
ee.Initialize()

## Import packages and the Canopy Nitrogen Content models

In [38]:
import CNCmodel_Cab
import CNCmodel_Cprot

## User definition: choose region of interest, timeframe and asset path

In [39]:
# Region of interest (MNI study site)
roi = ee.Geometry.Polygon(
    [[[11.67427222, 48.29512500],
      [11.67132778, 48.23868889],
      [11.73912778, 48.23709167],
      [11.74214722, 48.29352500]]])

# Start and end date and timesteps 
startDateStr = '2020-07-01'
endDateStr = '2020-08-31'
timeWindows = 7

# Your own Google Earth Engine asset path 
assetPath = 'projects/ee-declerck/assets/CNCjupyterTest/'

# Choose Cprot or Cab model
CNCmethod = 'Cprot'

## Set up variables and functions

In [41]:
# Parameter settings
startDateGEE = ee.Date(startDateStr)
startDateGEE = ee.Date(startDateStr)
endDate = datetime.datetime.strptime(endDateStr, '%Y-%m-%d').date()

variables_GREEN = {'CNCmethod':['CNCmethod', 25, 0.001]}

S2MSI = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

sys.setrecursionlimit(10000000)

# Cloud mask
def maskS2cloud(image):
  model = globals()['CNCmodel_' + CNCmethod]
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(model.scaleFactor_GREEN).copyProperties(qa).set('system:time_start', qa.get('system:time_start'))


# Desert and water masks
bare_cover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('bare-coverfraction').lte(90);
lakes = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').eq(80)
lakemask = lakes.eq(0);


# Some auxilary functions
def sequence_GREEN(CNCmethod):
    sequence_GREEN = []
    model = globals()['CNCmodel_' + CNCmethod]
    for i in range(0, model.XTrain_dim_GREEN):
        sequence_GREEN.append(str(i))
    return sequence_GREEN

def getInputDates(i):
  fecha_inicio = startDateGEE.advance(ee.Number(i).multiply(timeWindows),'day')
  fecha_fin = fecha_inicio.advance(timeWindows, 'day')
  fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value']/1000.0).strftime('%Y%m%d')
  return {'fecha_inicio':fecha_inicio, 'fecha_fin':fecha_fin, 'fecha_str':fecha_str}

def addVariables(image):
  date = ee.Date(image.get("system:time_start"));
  years = date.difference(ee.Date('1970-01-01'),'days');
  return image.addBands(ee.Image(years).rename('t').int());

## Implementation Gaussian Processes Regression to calculate CNC with the chosen method

In [42]:
def calculate_CNC(fecha_inicio, fecha_fin, CNCmethod):

  model = globals()['CNCmodel_' + CNCmethod]
  image = ee.Image(S2MSI
  .filterDate(fecha_inicio, fecha_fin)
  .filterBounds(roi)
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 40)
  .map(maskS2cloud)
  .select(model.bands)
  .max()
  .clip(roi));

  im_norm_ell2D_hypell = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).multiply(model.hyp_ell_GREEN).toArray().toArray(1);
  im_norm_ell2D = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5);

  PtTDX  = ee.Image(model.X_train_GREEN).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([sequence_GREEN(CNCmethod)]);
  arg1   = PtTPt.exp().multiply(model.hyp_sig_GREEN);
  k_star = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(model.alpha_coefficients_GREEN.toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[CNCmethod + '_CNC']]);
  mean_pred = mean_pred.add(model.mean_model_GREEN);

  k_star_uncert = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().multiply(arg1).toArray();
  Vvector = ee.Image(model.Linv_pre_calc_GREEN).matrixMultiply(k_star_uncert.toArray(0).toArray(1)).arrayProject([0])
  Variance = ee.Image(model.hyp_sig_unc_GREEN).toArray().subtract(Vvector.arrayDotProduct(Vvector)).abs().sqrt()
  Variance = Variance.toArray(1).arrayProject([0]).arrayFlatten([[CNCmethod + '_UNCERTAINTY_CNC']])

  image= image.addBands(mean_pred);
  image = image.addBands(Variance);

  return image.select(CNCmethod + '_CNC', CNCmethod + '_UNCERTAINTY_CNC')


# Function to loop calculate CNC through timeframe

In [43]:
def loop_CNC():
    startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
    daysIterations = abs((startDate - endDate) // timeWindows).days

    for i in range(0, daysIterations):
        print(getInputDates(i)['fecha_str'])
        imageHolder = ee.Image().set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows), 'days').millis())

        for variable_GREEN in variables_GREEN:
            params = variables_GREEN[variable_GREEN]
            variable = params[0]
            limitUp = params[1]
            limitDown = params[2]
            imagen = calculate_CNC(getInputDates(i)['fecha_inicio'], getInputDates(i)['fecha_fin'], CNCmethod)
            imageHolder = imageHolder.addBands(imagen)

            image_export = imageHolder.select(CNCmethod + '_CNC', CNCmethod + 'UNCERTAINTY_CNC')
            image_export = image_export.mask(lakemask)
            image_export = image_export.mask(bare_cover)
            image_export = image_export.set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows),'days').millis());

            exportar = ee.batch.Export.image.toAsset(
            assetId=assetPath + getInputDates(i)['fecha_str'] + CNCmethod + '_CNC',
            image=image_export,
            maxPixels=4731453308,
            description=getInputDates(i)['fecha_str'] + CNCmethod + '_CNC',
            scale=20,
            region=roi
            )
            exportar.start()
            exportar.status()

In [44]:
loop_CNC()

20200701
20200708
20200715
20200722
20200729
20200805
20200812
20200819
